# Warm-up
1. Review this code for 1 minute, then:
    1. Identify how an "Electric-type" Pokemon object would get access to its base statistics
    1. Attempt to write a method for `Electric` that will check its `HP` after every action

<img src='../assets/inherit_warmup.png' width=500 align='left' />

---
# Learning Objectives

1. Students will be able to visually identify `class` inheritance
1. Students will be able to write basic `class`es that inherit from others
1. Students will be able to use basic decorators to create `dataclasses`

---
# Object-Oriented Programming Seminar: Expanding Classes
The last major lesson in OOP is class inheritance. Class inheritance is the act of one object "gaining" all of the functionality of another object. [GeeksforGeeks](https://www.geeksforgeeks.org/inheritance-in-python/) states that the main purposes of class inheritance are:
> 1. Represents real-world relationships well
> 1. Provides reusability of code
> 1. It is transitive in nature

## The Big Idea
Any object in Python worth anything should exercise the use of inheritance because it allows for **extensibility**, **reusability**, and _clarity_. Just like a single function should do a single job, a single `class` should do a specific thing. However, we have already expanded the work of a single function before by using nested functions (a function that calls another function). Likewise, we can expand a `class` by "nesting" it with other `class`es.

<img src='../assets/nourdine-diouane-4YJkvZGDcyU-unsplash.jpg' width=700/>

---
# Last Class
For a quick reminder of where we left off last class

In [1]:
import class_demo as demo

In [2]:
%psource demo.Pileup

class Pileup:
    """An object that represents the observed bases and their counts at a specific position
    
    Attributes:
        depth_offset (int): how depth should be offset for normalization (default: 0)
        counts (collections.Counter): Observed bases and their number of occurrences (default: None)
        depth (int): Sum of all observed base counts
        consensus (collections.namedtuple or None): The most common base and its number of occurrences
        maf (float or None): the mean allele frequency of the consensus base versus depth
    """
    
    __slots__ = 'depth_offset _counts'.split()
    
    def __init__(self, depth_offset = 0, counts = None):
        self.depth_offset = depth_offset
        if counts is None:
            self._counts = Counter()
        else:
            if isinstance(counts, Counter):
                self._counts = counts
            else:
                raise ValueError('counts must be a collections.Counter')
    
    @property
    def

---
# Class Inheritance

Class inheritance is when one object takes/gives attributes and methods to another object upon its instantiation.

<img src='../assets/pokegeny.jpg' />

[Shelomi et al. 2012. A Phylogeny and Evolutionary History of the Pokémon. Annals of Improbable Research](../assets/Phylogeny-Pokemon.pdf)

## Salient Functions

In [3]:
def generate_random_integers(total, n):
    """Generates a list of n integers that sum up to a given number
    
    Adapted from http://sunny.today/generate-random-integers-with-fixed-sum/
    
    Args:
        total (int): the total all the integers are to sum up to
        n (int): the number of integers
    
    Returns:
        (list): a list if integers that sum approximately to total
    """
    μ = total / n 
    var = int(0.25 * μ)

    min_v = μ - var
    max_v = μ + var
    vals = [min_v] * n

    diff = total - min_v * n
    while diff > 0:
        a = random.randint(0, n - 1)
        if vals[a] >= max_v:
            continue
        vals[a] += 1
        diff -= 1
    return [int(val) for val in vals]

---
# Let's play with the data

In [4]:
import pandas as pd

In [5]:
# Read in the pokemon csv
pokedex = pd.read_csv('../datasets/pokemon.csv')

In [6]:
# Show just Pichu's data
pokedex[pokedex.Name == 'Pichu']

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
186,172,Pichu,Electric,NaN,205,20,40,15,35,35,60,2,False


What is Pichu's type?

## Exploring class inheritance by doing something productive: making Pokemon&copy;

In [7]:
# Base class of Pokemon
class Pokemon:
    def __init__(self, level = 1, name = None, given_name = None):
        self.level = level
        self.given_name = given_name
        pokedex = pd.read_csv('../datasets/pokemon.csv')
        self.name = name.title() if name else None
        
        if name is None:
            self.base_hp,       \
            self.base_attack,   \
            self.base_defense,  \
            self.base_sAttack,  \
            self.base_sDefense, \
            self.base_speed = generate_random_integers(random.randint(125, 400), 6)
        
        elif pokedex.Name.str.contains(self.name).any():
            self.base_hp,       \
            self.base_attack,   \
            self.base_defense,  \
            self.base_sAttack,  \
            self.base_sDefense, \
            self.base_speed = pokedex.loc[pokedex.Name == self.name, [
                'HP', 
                'Attack', 
                'Defense', 
                'Sp. Atk', 
                'Sp. Def', 
                'Speed'
            ]].values[0]
        
        else:
            raise ValueError('unregistered Pokemon')
        
        self.current_hp = self.base_hp
        self.exp = 0
    
    def __str__(self):
        return f'Pokemon(level = {self.level}, name = {self.given_name if self.given_name else self.name if self.name else "MISSINGNO"})'
    
    def __repr__(self):
        return f'Pokemon(level = {self.level}, name = {self.name}, given_name = {self.given_name})'
    
    def stats(self):
        return pd.Series([self.base_hp, self.base_attack, self.base_defense, self.base_sAttack, self.base_sDefense, self.base_speed],
                           index = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed'])

In [8]:
magikarp = Pokemon(name='Magikarp')

In [10]:
print(magikarp)
print(repr(magikarp))
magikarp.stats()

Pokemon(level = 1, name = Magikarp)
Pokemon(level = 1, name = Magikarp, given_name = None)


HP         20
Attack     10
Defense    55
Sp. Atk    15
Sp. Def    20
Speed      80
dtype: int64

In [11]:
class Electric(Pokemon):
    def __init__(self, level = 1, name = None, given_name = None):
        Pokemon.__init__(self, level, name, given_name)
        self.type = 'Electric'
        self.weak_def = ('Ground')
        self.half_def = ('Electric', 'Flying')
        self.strong_att = ('Flying', 'Water')
        self.half_att = ('Dragon', 'Electric', 'Grass')
        self.no_att = ('Ground')
        self.immune = ('Paralyze')
        
    def __repr__(self):
        return super().__repr__().replace('Pokemon', 'Electric')
    
    def __str__(self):
        return super().__str__().replace('Pokemon', 'Electric')

In [12]:
pichu = Electric(name='Pichu')

In [14]:
pichu.immune

'Paralyze'

---
# Workshop
In groups of 3-4 people:
   * Identify the different "types" of Pokemon (not including Electric)
   * Choose one "type" (cannot be Electric)
   * Write your own "type" subclass

---

In [ ]:
class Pichu(Electric):
    def __init__(self, level = 1, name = 'Pichu', given_name = None):
        Electric.__init__(self, level, name, given_name)
        self.name = name.title()
        
    def __repr__(self):
        return super().__repr__().replace('Electric', 'Pichu')
    
    def __str__(self):
        return super().__str__().replace('Electric', 'Pichu')
    
    def thunder_shock(self):
        ability_type = 'Electric'
        self.thunder_shock_pp = 30
        power = 40
        accuracy = 1
        effect = ('Paralyze', .1)
        
        return (ability_type, effect, accuracy * power * self.base_sAttack)
    
    def charm(self):
        ability_type = 'Fairy'
        self.charm_pp = 20
        power = None
        accuracy = None
        effect = ('Decrease_Attack', 1)
        
        return (ability_type, effect, None)
    
    def tail_whip(self):
        if self.level >= 5:
            ability_type = None
            self.tail_whip_pp = 30
            power = 1
            accuracy = 1
            effect = None

            return (ability_type, effect, accuracy * power * self.base_attack)

        else:
            raise IndexError('Move not available yet')
        
    def sweet_kiss(self):
        if self.leve >= 10:
            ability_type = 'Fairy'
            self.sweet_kiss_pp = 10
            power = None
            accuracy = None
            effect = ('Confusion', .75)
            
            return (ability_type, effect, None)
        
        else:
            raise IndexError('Move not available yet')
        
    def nasty_plot(self):
        if self.level >= 13:
            ability_type = 'Dark'
            self.nasty_plot_pp = 20
            power = None
            accuracy = None
            effect = ('Decrease_sAttack', 1)
            
            return (ability_type, effect, None)
        else:
            raise IndexError('Move not available yet')

    def thunder_wave(self):
        if self.level >= 18:
            ability_type = 'Electric'
            self.thunder_wave_pp = 20
            power = 40
            accuracy = 0.9
            effect = ('Paralyze', 1)

            return(ability_type, effect, accuracy * power * self.base_sAttack)
        
        else:
            raise IndexError('Move not available yet')